There are a few simple options for modelling ranking:

1. If you observe the result, not just the winner, but the result (e.g., 4-1), you could model the difference (e.g., +3) as a Gaussian variable depending on Gaussian distributed player skills.

2. If you only observe the winner, but don't know how tight the match was, then the outcome should indeed be modelled as a bernoulli/categorical variable.

  - If you want to model player skills as Gaussian variables, this would require a new node in BayesPy which converts a Gaussian variable to Bernoulli variable via `>` comparison. It would be definitely an interesting feature and I'd love to work on that but unfortunately I don't have the time for that.
  
  - With current nodes, it *might* be possible to find maximum likelihood estimates for the player skills. Not sure though, would need to look a bit more into it. You would just create some point estimate node for player skills, do some mapping to get match outcome probabilities and then feed that to Bernoulli node.

You were asking about (2), but I'll show how you could do (1) in case you're interested:

In [8]:
import bayespy as bp

Player skills are Gaussian variables:

In [9]:
players = bp.nodes.GaussianARD(
    0,
    1e-6,
    shape=(4,),
    name="players"
)

Note that I'm using `GaussianARD` because the skills are scalars, but I still have them in the same node so that their posterior couplings can be learned. Using `shape` instead of `plates` for that.

Form teams for each match. You were interested in 1-on-1 matches, so just put `+1` and `-1` values for the active players. But in general, some approaches:

- If team total skill is a sum of player skills: `+1` for the home team, `-1` for the away team, `0` for non-playing players
- If teams can have different number of players and increasing the player count in a team has smaller than linear effect, then perhaps weights: `1/sqrt(N_home)` and `-1/sqrt(N_away)`.
- If team total skill is just an average of the player skills: `1/N_home` and `-1/N_away`.

The match outcome is then assumed to depend on the team total skill differences, therefore the different signs `+` and `-`.

In [10]:
matches = bp.nodes.SumMultiply(
    "i,i",
    players,
    [
        [1, -1, 0, 0],
        [1, 0, -1, 0],
        [1, 0, 0, -1],
        [0, 1, -1, 0],
        [0, 1, 0, -1],
        [0, 0, 1, -1],
    ],
    name="matches",
)

We assume that we observe some measure of team skill differences. In football, it could be the difference between the scores (4 - 1 = +3). Outcome is always a bit noisy, so use some fixed noise parameter or learn that parameter too.

In [11]:
results = bp.nodes.GaussianARD(
    matches,
    1e-1,
    name="results",
)

Positive value means that the home team won. Negative values means that the away team won.

In [12]:
results.observe([1-4, 2-3, 2-1, 3-2, 5-1, 2-0])

Update the only unknown variables and show the posterior estimates of player skills.

In [13]:
players.update()

print(players)

players ~ Gaussian(mu, Cov)
  mu = 
[-0.74999813  1.999995    0.49999875 -1.74999563]
  Cov = 
[[250001.87500356 249999.37500981 249999.37500981 249999.37500981]
 [249999.37500981 250001.87500356 249999.37500981 249999.37500981]
 [249999.37500981 249999.37500981 250001.87500356 249999.37500981]
 [249999.37500981 249999.37500981 249999.37500981 250001.87500356]]



Note that the posterior covariance is huge because there's no anchor point (i.e., we could translate the skills arbitrarily because we are only interested in skill differences), but there's also a huge correlation between the skills so the player skill order can be quite certain although the absolute skill values are uncertain.